# TSA Chapter 5: GARCH Volatility Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_garch.ipynb)

This notebook demonstrates:
- Volatility clustering in financial returns
- ARCH/GARCH model specification and estimation
- News impact curves
- Volatility forecasting and Value at Risk (VaR)

In [ ]:
!pip install numpy pandas matplotlib scipy arch yfinance -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from arch import arch_model
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Style configuration
COLORS = {"blue": "#1A3A6E", "red": "#DC3545", "green": "#2E7D32",
          "orange": "#E67E22", "gray": "#666666", "purple": "#8E44AD"}

plt.rcParams.update({
    "axes.facecolor": "none", "figure.facecolor": "none",
    "savefig.transparent": True, "axes.spines.top": False,
    "axes.spines.right": False, "axes.grid": False,
    "font.size": 9, "axes.titlesize": 10, "axes.labelsize": 9,
    "figure.dpi": 150, "lines.linewidth": 1.2,
})

def save_chart(fig, name):
    fig.savefig(f"{name}.pdf", bbox_inches="tight", transparent=True, dpi=150)
    fig.savefig(f"{name}.png", bbox_inches="tight", transparent=True, dpi=150)
    try:
        charts_path = os.path.join("..", "..", "charts", name)
        fig.savefig(f"{charts_path}.pdf", bbox_inches="tight", transparent=True, dpi=150)
        fig.savefig(f"{charts_path}.png", bbox_inches="tight", transparent=True, dpi=150)
    except Exception: pass
    print(f"Saved: {name}.pdf + .png")

In [ ]:
# Download BTC data
btc = yf.download("BTC-USD", start="2019-01-01", progress=False)
returns = btc["Close"].pct_change().dropna() * 100
print(f"BTC returns: {len(returns)} obs, mean={returns.mean():.4f}%, std={returns.std():.4f}%")
print(f"Skewness: {returns.skew():.4f}, Kurtosis: {returns.kurtosis():.4f}")

In [ ]:
# Chart 1: Volatility clustering
fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(returns.index, returns.values, color=COLORS["blue"], linewidth=0.5)
ax.axhline(y=0, color="black", linewidth=0.5, alpha=0.3)
ax.set_title("Bitcoin Daily Returns: Volatility Clustering", fontweight="bold")
ax.set_ylabel("Return (%)")
ax.legend(["BTC daily returns (%)"], loc="upper center",
          bbox_to_anchor=(0.5, -0.12), frameon=False)
fig.tight_layout()
save_chart(fig, "ch5_volatility_clustering")
plt.show()

In [ ]:
# Chart 2: GARCH(1,1) estimation
am = arch_model(returns, vol="Garch", p=1, q=1, dist="t")
res = am.fit(disp="off")
print(res.summary())

fig, axes = plt.subplots(2, 1, figsize=(10, 5))
axes[0].plot(returns.index, returns.values, color=COLORS["blue"], lw=0.5, alpha=0.7)
axes[0].set_title("BTC Returns", fontweight="bold")
axes[0].set_ylabel("Return (%)")

cond_vol = res.conditional_volatility
axes[1].plot(cond_vol.index, cond_vol.values, color=COLORS["red"], lw=0.8)
axes[1].set_title("GARCH(1,1) Conditional Volatility", fontweight="bold")
axes[1].set_ylabel("$\sigma_t$ (%)")
fig.tight_layout()
save_chart(fig, "ch5_garch_fit")
plt.show()

In [ ]:
# Chart 3: News Impact Curve
omega = res.params["omega"]
alpha = res.params["alpha[1]"]
beta = res.params["beta[1]"]
sigma2_bar = omega / (1 - alpha - beta)

eps = np.linspace(-10, 10, 200)
nic_garch = omega + alpha * eps**2 + beta * sigma2_bar

fig, ax = plt.subplots(figsize=(7, 3.5))
ax.plot(eps, nic_garch, color=COLORS["blue"], lw=1.5, label="GARCH(1,1)")
ax.axvline(0, color=COLORS["gray"], ls="--", lw=0.5)
ax.set_xlabel(r"$\varepsilon_{t-1}$")
ax.set_ylabel(r"$\sigma_t^2$")
ax.set_title("News Impact Curve", fontweight="bold")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), frameon=False)
fig.tight_layout()
save_chart(fig, "ch5_news_impact")
plt.show()